# Plotly 图表

`PnPlotly` 组件允许在 Panel 应用程序中显示 [Plotly 图表](https://plotly.com/python/)。它通过对 Plotly 对象中包含的数组数据使用二进制序列化来提高图表更新速度。

请注意，要在 Jupyter 笔记本中使用 Plotly 组件，必须激活 Panel 扩展并包含 `"plotly"` 作为参数。这一步确保正确设置 plotly.js。

底层实现为`panel.pane.Plotly`，参数基本一致，参考文档：https://panel.holoviz.org/reference/panes/Plotly.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

让我们创建一个基本示例：

创建后，`PnPlotly` 组件可以通过分配新的图形对象来更新：

In [2]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnButton @click="update_fig()">Update</PnButton>
  <PnPlotly :object="fig.value"/>
</template>
<script lang='py'>
from vuepy import ref
import numpy as np
import plotly.graph_objs as go

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))

surface = go.Surface(z=z)
fig = go.Figure(data=[surface])

fig.update_layout(
    title="Plotly 3D",
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50),
)
fig = ref(fig)

def update_fig():
    new_fig = go.Figure(data=[go.Surface(z=np.sin(z+1))])
    new_fig.update_layout(
        title="Update Plotly 3D",
        width=500,
        height=500,
        margin=dict(t=50, b=50, r=50, l=50),
    )
    fig.value = new_fig
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnButton @click=\"update_fig()\">Update</PnButton>\n  <PnPlotly :object=\"fig.value\"/>\n</template>\n<script lang='py'>\nfrom vuepy import ref\nimport numpy as np\nimport plotly.graph_objs as go\n\nxx = np.linspace(-3.5, 3.5, 100)\nyy = np.linspace(-3.5, 3.5, 100)\nx, y = np.meshgrid(xx, yy)\nz = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))\n\nsurface = go.Surface(z=z)\nfig = go.Figure(data=[surface])\n\nfig.update_layout(\n    title=\"Plotly 3D\",\n    width=500,\n    height=500,\n    margin=dict(t=50, b=50, r=50, l=50),\n)\nfig = ref(fig)\n\ndef update_fig():\n    new_fig = go.Figure(data=[go.Surface(z=np.sin(z+1))])\n    new_fig.update_layout(\n        title=\"Update Plotly 3D\",\n        width=500,\n        height=500,\n        margin=dict(t=50, b=50, r=50, l=50),\n    )\n    fig.value = new_fig\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] Button(name='Update')
                [1] Plotly(Figure)


## 布局示例

`PnPlotly` 组件支持任意复杂度的布局和子图，允许显示即使是深度嵌套的 Plotly 图形：


In [4]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnPlotly :object="fig_layout" />
</template>
<script lang='py'>
import numpy as np
import plotly.graph_objs as go
from plotly import subplots

heatmap = go.Heatmap(
    z=[[1, 20, 30],
       [20, 1, 60],
       [30, 60, 1]],
    showscale=False)

y0 = np.random.randn(50)
y1 = np.random.randn(50)+1

box_1 = go.Box(y=y0)
box_2 = go.Box(y=y1)
data = [heatmap, box_1, box_2]

fig_layout = subplots.make_subplots(
    rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
    subplot_titles=('first subplot','second subplot', 'third subplot')
)

fig_layout.append_trace(box_1, 1, 1)
fig_layout.append_trace(box_2, 1, 2)
fig_layout.append_trace(heatmap, 2, 1)

fig_layout['layout'].update(height=600, width=600, title='i <3 subplots')
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnPlotly :object=\"fig_layout\" />\n</template>\n<script lang='py'>\nimport numpy as np\nimport plotly.graph_objs as go\nfrom plotly import subplots\n\nheatmap = go.Heatmap(\n    z=[[1, 20, 30],\n       [20, 1, 60],\n       [30, 60, 1]],\n    showscale=False)\n\ny0 = np.random.randn(50)\ny1 = np.random.randn(50)+1\n\nbox_1 = go.Box(y=y0)\nbox_2 = go.Box(y=y1)\ndata = [heatmap, box_1, box_2]\n\nfig_layout = subplots.make_subplots(\n    rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],\n    subplot_titles=('first subplot','second subplot', 'third subplot')\n)\n\nfig_layout.append_trace(box_1, 1, 1)\nfig_layout.append_trace(box_2, 1, 2)\nfig_layout.append_trace(heatmap, 2, 1)\n\nfig_layout['layout'].update(height=600, width=600, title='i <3 subplots')\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Plotly(Figure)


## 响应式图表

通过在 Plotly 布局上使用 `autosize` 选项和 `PnPlotly` 组件的响应式 `sizing_mode` 参数，可以使 Plotly 图表具有响应性：


In [5]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnCol name="## A responsive plot" sizing_mode="stretch_width">
    <PnPlotly :object="fig_responsive" :height="300" sizing_mode="stretch_width" />
  </PnCol>
</template>
<script lang='py'>
import pandas as pd
import plotly.express as px

data = pd.DataFrame([
    ('Monday', 7), ('Tuesday', 4), ('Wednesday', 9), ('Thursday', 4),
    ('Friday', 4), ('Saturday', 4), ('Sunday', 4)], columns=['Day', 'Orders']
)

fig_responsive = px.line(data, x="Day", y="Orders")
fig_responsive.update_traces(mode="lines+markers", marker=dict(size=10), line=dict(width=4))
fig_responsive.layout.autosize = True
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnCol name=\"## A responsive plot\" sizing_mode=\"stretch_width\">\n    <PnPlotly :object=\"fig_responsive\" :height=\"300\" sizing_mode=\"stretch_width\" />\n  </PnCol>\n</template>\n<script lang='py'>\nimport pandas as pd\nimport plotly.express as px\n\ndata = pd.DataFrame([\n    ('Monday', 7), ('Tuesday', 4), ('Wednesday', 9), ('Thursday', 4),\n    ('Friday', 4), ('Saturday', 4), ('Sunday', 4)], columns=['Day', 'Orders']\n)\n\nfig_responsive = px.line(data, x=\"Day\", y=\"Orders\")\nfig_responsive.update_traces(mode=\"lines+markers\", marker=dict(size=10), line=dict(width=4))\nfig_responsive.layout.autosize = True\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column(name='## A responsive plot', sizing_mode='stretch_width')
                [0] Plotly(Figure, height=300, sizing_mode='stretch_width')


## 图表配置

您可以通过 `config` 参数设置 [Plotly 配置选项](https://plotly.com/javascript/configuration-options/)。让我们尝试配置 `scrollZoom`：


In [6]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnCol name="## A responsive and scroll zoomable plot" 
         sizing_mode="stretch_width">
    <PnPlotly 
      :object="fig_responsive" 
      :config="{'scrollZoom': True}" 
      :height="300" 
      sizing_mode="stretch_width" />
  </PnCol>
</template>
<script lang='py'>
import pandas as pd
import plotly.express as px

data = pd.DataFrame([
    ('Monday', 7), ('Tuesday', 4), ('Wednesday', 9), ('Thursday', 4),
    ('Friday', 4), ('Saturday', 4), ('Sunday', 4)], columns=['Day', 'Orders']
)

fig_responsive = px.line(data, x="Day", y="Orders")
fig_responsive.update_traces(mode="lines+markers", marker=dict(size=10), line=dict(width=4))
fig_responsive.layout.autosize = True
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnCol name=\"## A responsive and scroll zoomable plot\" \n         sizing_mode=\"stretch_width\">\n    <PnPlotly \n      :object=\"fig_responsive\" \n      :config=\"{'scrollZoom': True}\" \n      :height=\"300\" \n      sizing_mode=\"stretch_width\" />\n  </PnCol>\n</template>\n<script lang='py'>\nimport pandas as pd\nimport plotly.express as px\n\ndata = pd.DataFrame([\n    ('Monday', 7), ('Tuesday', 4), ('Wednesday', 9), ('Thursday', 4),\n    ('Friday', 4), ('Saturday', 4), ('Sunday', 4)], columns=['Day', 'Orders']\n)\n\nfig_responsive = px.line(data, x=\"Day\", y=\"Orders\")\nfig_responsive.update_traces(mode=\"lines+markers\", marker=dict(size=10), line=dict(width=4))\nfig_responsive.layout.autosize = True\n</script>\n", "setup": ""}


Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column(name='## A responsive a..., sizing_mode='stretch_width')
                [0] Plotly(Figure, config={'scrollZoom': True}, height=300, sizing_mode='stretch_width')


尝试在图表上用鼠标滚动！

## 增量更新

您可以通过使用字典而不是 Plotly Figure 对象来高效地增量更新轨迹或布局，而不是更新整个 Figure。

请注意，增量更新只有在将 `Figure` 定义为字典时才会高效，因为 Plotly 会复制轨迹，这意味着原地修改它们没有效果。修改数组将仅发送该数组（使用二进制协议），从而实现快速高效的更新。


In [6]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnPlotly :object="fig_patch" ref="plotly_pane_patch" />
  <PnRow>
    <PnButton @click="update_z()">更新数据</PnButton>
    <PnButton @click="update_layout()">更新布局</PnButton>
    <PnButton @click="reset()">重置</PnButton>
  </PnRow>
</template>
<script lang='py'>
import numpy as np
import plotly.graph_objs as go
from vuepy import ref

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))

surface = go.Surface(z=z)
layout = go.Layout(
    title='Plotly 3D 图表',
    autosize=False,
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50)
)

fig_patch = dict(data=[surface], layout=layout)
plotly_pane_patch = ref(None)

def update_z():
    surface.z = np.sin(z+1)
    plotly_pane_patch.value.unwrap().object = fig_patch
    
def update_layout():
    fig_patch['layout']['width'] = 800
    plotly_pane_patch.value.unwrap().object = fig_patch
    
def reset():
    surface.z = z
    fig_patch['layout']['width'] = 500
    plotly_pane_patch.value.unwrap().object = fig_patch
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnPlotly :object=\"fig_patch\" ref=\"plotly_pane_patch\" />\n  <PnRow>\n    <PnButton @click=\"update_z()\">\u66f4\u65b0\u6570\u636e</PnButton>\n    <PnButton @click=\"update_layout()\">\u66f4\u65b0\u5e03\u5c40</PnButton>\n    <PnButton @click=\"reset()\">\u91cd\u7f6e</PnButton>\n  </PnRow>\n</template>\n<script lang='py'>\nimport numpy as np\nimport plotly.graph_objs as go\nfrom vuepy import ref\n\nxx = np.linspace(-3.5, 3.5, 100)\nyy = np.linspace(-3.5, 3.5, 100)\nx, y = np.meshgrid(xx, yy)\nz = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))\n\nsurface = go.Surface(z=z)\nlayout = go.Layout(\n    title='Plotly 3D \u56fe\u8868',\n    autosize=False,\n    width=500,\n    height=500,\n    margin=dict(t=50, b=50, r=50, l=50)\n)\n\nfig_patch = dict(data=[surface], layout=layout)\nplotly_pane_patch = ref(None)\n\ndef update_z():\n    surface.z = np.sin(z+1)\n    plotly_p

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Column
                [0] Plotly(dict)
                [1] Row
                    [0] Button(name='更新数据')
                    [1] Button(name='更新布局')
                    [2] Button(name='重置')


## 事件处理

`PnPlotly` 组件提供对 [Plotly 事件](https://plotly.com/javascript/plotlyjs-events/)的访问，如点击、悬停和选择(使用`Box Select`、`Lasso Select`工具)等：


In [7]:
%%vuepy_run --plugins vpanel --show-code --codegen-backend='panel'
<template>
  <PnPlotly :object="fig" ref="plotly_ref" 
            @click='on_click'
            @selected='on_selected'
  />
</template>
<script lang='py'>
import numpy as np
import plotly.express as px
from vuepy import ref, onMounted

# 创建一些示例数据
df = px.data.iris()

# 创建散点图
fig = px.scatter(
    df, x="sepal_width", y="sepal_length", 
    color="species", size="petal_length",
    hover_data=["petal_width"]
)

# 事件数据引用
click_data = ref({})
hover_data = ref({})
plotly_ref = ref(None)

def on_click(event):
    if not event:
        return
    print(event.new['points']) # [{'curveNumber': 2, 'pointNumber': 31, 
                               #   'pointIndex': 31, 'x': 3.8, 'y': 7.9, 
                               #   'marker.size': 6.4, 'customdata': [2]}]
    click_data.value = event.new['points']
    
def on_selected(event):
    if not event:
        return
    print(event.new['points']) # [{'curveNumber': 2, 'pointNumber': 31, ...
</script>

{"vue": "<!-- --plugins vpanel --show-code --codegen-backend='panel' -->\n<template>\n  <PnPlotly :object=\"fig\" ref=\"plotly_ref\" \n            @click='on_click'\n            @selected='on_selected'\n  />\n</template>\n<script lang='py'>\nimport numpy as np\nimport plotly.express as px\nfrom vuepy import ref, onMounted\n\n# \u521b\u5efa\u4e00\u4e9b\u793a\u4f8b\u6570\u636e\ndf = px.data.iris()\n\n# \u521b\u5efa\u6563\u70b9\u56fe\nfig = px.scatter(\n    df, x=\"sepal_width\", y=\"sepal_length\", \n    color=\"species\", size=\"petal_length\",\n    hover_data=[\"petal_width\"]\n)\n\n# \u4e8b\u4ef6\u6570\u636e\u5f15\u7528\nclick_data = ref({})\nhover_data = ref({})\nplotly_ref = ref(None)\n\ndef on_click(event):\n    if not event:\n        return\n    print(event.new['points']) # [{'curveNumber': 2, 'pointNumber': 31, \n                               #   'pointIndex': 31, 'x': 3.8, 'y': 7.9, \n                               #   'marker.size': 6.4, 'customdata': [2]}]\n    click_data.val

Column(name='root')
    [0] Column(name='body')
        [0] Column
            [0] Plotly(Figure)


## API

### 属性

| 属性名                      | 说明                          | 类型                                                           | 默认值 |
| -------------------------- | ----------------------------- | ---------------------------------------------------------------| ------- |
| object                     | 正在显示的 Plotly `Figure` 或字典对象 | ^[object]                                              | None |
| config                     | 图表的额外配置。参见 [Plotly 配置选项](https://plotly.com/javascript/configuration-options/) | ^[dict] | {} |
| link_figure                | 当 Plotly `Figure` 原地修改时更新显示的 Plotly 图表 | ^[boolean]                            | True |
| click_data                 | 来自 `plotly_click` 事件的点击事件数据 | ^[dict]                                                | {} |
| clickannotation_data       | 来自 `plotly_clickannotation` 事件的点击注释事件数据 | ^[dict]                           | {} |
| hover_data                 | 来自 `plotly_hover` 和 `plotly_unhover` 事件的悬停事件数据 | ^[dict]                       | {} |
| relayout_data              | 来自 `plotly_relayout` 事件的重新布局事件数据 | ^[dict]                                   | {} |
| restyle_data               | 来自 `plotly_restyle` 事件的重新样式事件数据 | ^[dict]                                    | {} |
| selected_data              | 来自 `plotly_selected` 和 `plotly_deselect` 事件的选择事件数据 | ^[dict]                  | {} |
| viewport                   | 当前视口状态，即显示图表的 x 和 y 轴限制。在 `plotly_relayout`、`plotly_relayouting` 和 `plotly_restyle` 事件时更新 | ^[dict] | {} |
| viewport_update_policy     | 用户交互期间更新视口参数的策略 | ^[str]                                                        | 'mouseup' |
| viewport_update_throttle   | 当 viewport_update_policy 为 "throttle" 时，视口更新同步的时间间隔（毫秒） | ^[int]              | 200 |
| sizing_mode                | 尺寸调整模式                   | ^[str]                                                         | 'fixed'  |
| width                      | 宽度                          | ^[int, str]                                                    | None    |
| height                     | 高度                          | ^[int, str]                                                    | None    |
| min_width                  | 最小宽度                      | ^[int]                                                         | None    |
| min_height                 | 最小高度                      | ^[int]                                                         | None    |
| max_width                  | 最大宽度                      | ^[int]                                                         | None    |
| max_height                 | 最大高度                      | ^[int]                                                         | None    |
| margin                     | 外边距                        | ^[int, tuple]                                                  | 5       |
| css_classes                | CSS类名列表                   | ^[list]                                                        | []      |

### Events

| 事件名 | 说明                  | 类型                                   |
| ---   | ---                  | ---                                    |
| click  | 当元素被点击时触发的事件 | ^[Callable]`(Event) -> None`    |
| hover | 当元素被鼠标覆盖时触发 | ^[Callable]`(Event) -> None`    |
| selected | 当元素被`Box Select`、`Lasso Select`工具选中时触发 | ^[Callable]`(Event) -> None`    |
| doubleclick  | 当元素被双击时触发的事件 | ^[Callable]`(Event) -> None`    |
| clickannotation | 当元素被鼠标覆盖时触发 | ^[Callable]`(Event) -> None`    |

### Slots

| 插槽名   | 说明               |
| ---     | ---               |
| default | 自定义默认内容      |

### 方法

| 属性名 | 说明 | 类型 |
| --- | --- | --- |


In [8]:
##ignore
import numpy as np
import plotly.graph_objs as go
import panel as pn

pn.extension("plotly")

xx = np.linspace(-3.5, 3.5, 100)
yy = np.linspace(-3.5, 3.5, 100)
x, y = np.meshgrid(xx, yy)
z = np.exp(-((x - 1) ** 2) - y**2) - (x**3 + y**4 - x / 5) * np.exp(-(x**2 + y**2))

surface=go.Surface(z=z)
fig = go.Figure(data=[surface])

fig.update_layout(
    title="Plotly 3D Plot",
    width=500,
    height=500,
    margin=dict(t=50, b=50, r=50, l=50),
)

plotly_pane = pn.pane.Plotly(fig)
# dir(plotly_pane.param)
plotly_pane.controls()

Tabs
    [0] WidgetBox(margin=(5, 10), name='Controls')
        [0] StaticText(value='<b>Controls</b>')
        [1] DictInput(description='Plotly configuration o..., name='Config', serializer='json', type=<class 'dict'>)
        [2] DictInput(description='Relayout event d..., name='Relayout data', serializer='json', type=<class 'dict'>)
        [3] ListInput(description='Restyle event d..., name='Restyle data', serializer='json', type=<class 'list'>)
        [4] DictInput(description='Current viewport state, ..., name='Viewport', serializer='json', type=<class 'dict'>)
        [5] Select(description='Policy by which the viewp..., name='Viewport update policy', options=OrderedDict([('mouseup', ...]), value='mouseup')
        [6] IntInput(description='Time interval i..., name='Viewport update t..., start=0, value=200)
    [1] WidgetBox(margin=(5, 10), name='Layout')
        [0] StaticText(value='<b>Layout</b>')
        [1] TextInput(description='String identifier f..., name='Name', value='Plotly00216')
        [2] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
        [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
        [4] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
        [5] IntInput(description='The height of the compone..., name='Height', start=0)
        [6] IntInput(description='Minimal width o..., name='Min width', start=0)
        [7] IntInput(description='Minimal height o..., name='Min height', start=0)
        [8] IntInput(description='Maximum width o..., name='Max width', start=0)
        [9] IntInput(description='Maximum height o..., name='Max height', start=0)
        [10] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
        [11] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
        [12] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
        [13] IntInput(description='The width of the componen..., name='Width', start=0)
        [14] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
        [15] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
        [16] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]))
        [17] Checkbox(name='Visible', value=True)